In [1]:
import os
import sys
import time
from pathlib import Path

nb_dir = os.path.split(os.getcwd())[0]

sys.path.append(nb_dir)
sys.path.append(os.path.split(nb_dir)[0])

In [2]:
import pandas as pd
from PIL import Image, ImageDraw
import copy

In [3]:
from services.xml_document_info import (get_xml_info, get_xml_image_info)
from services.get_xml import  create_pdf_processing_paths, extract_pdf_metadata, process_input_pdf
from anuvaad_auditor.loghandler import log_info
from anuvaad_auditor.loghandler import log_error
from src.services import main
from src.services.get_underline import get_underline
from services import get_xml
from src.services.child_text_unify_to_parent import ChildTextUnify
from services.preprocess import prepocess_pdf_regions
from services.get_tables import page_num_correction , get_text_table_line_df
from src.services.ocr_text_utilities import  tesseract_ocr
from src.services.get_response import process_image_df,  process_table_df, df_to_json, process_line_df, process_bg_image

from utilities.filesystem import (create_directory)
import config

import src.utilities.app_context as app_context
app_context.init()

In [4]:
def draw_box(filepath, desired_width, desired_height, df, color="green", save=False):
    image  = Image.open(filepath)
    image  = image.resize((desired_width, desired_height))
    draw   = ImageDraw.Draw(image)
    
    for index, row in df.iterrows():
        left   = int(row['text_left'])
        right  = int(row['text_width'] + left)
        top    = int(row['text_top'])
        bottom = int(row["text_height"] + top)
        
        draw.rectangle(((left, top), (right,bottom)), outline=color)
    save_filepath = os.path.join(os.path.dirname(filepath), 'processed_' + os.path.basename(filepath))
    if save:
        image.save(save_filepath)
    
    return image

def show_df(df):
    return df.head(df.shape[0])

## start of the program

In [5]:
'''
  folder structure of test data goes like this
  - notebooks
      - sample-data
          - input
          - output
          
  the pdfs are present in "input" directory and they are *.pdf is added into .gitignore
  just to save repo size.
'''

base_dir   = os.getcwd()
input_dir  = os.path.join(base_dir, 'sample-data','input')
save_dir   = os.path.join(base_dir, 'sample-data', 'bbox_output')
output_dir = os.path.join(base_dir, 'sample-data', 'output')

filename   = '20.03.2020.pdf'
# filename  = 'Madras_HC_02.pdf'


In [6]:
from src.services import main

In [7]:
app_context.application_context = dict({'task_id': 'BM-DUMMY'})
pdf_path   = os.path.join(input_dir, filename)

data       = main.DocumentStructure(app_context, filename, "en", base_dir=input_dir)


[2020-09-09 12:51:29,384] {loghandler.py:48} MainThread DEBUG in loghandler: Block merger starting processing <module 'src.utilities.app_context' from '/Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/utilities/app_context.py'>
[2020-09-09 12:51:29,386] {loghandler.py:20} MainThread INFO in loghandler: document preprocessing started ===>
[2020-09-09 12:51:29,396] {loghandler.py:20} MainThread INFO in loghandler: created processing directories successfully /Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input/20.03.2020_144883b8-f26d-11ea-a75f-acde48001122
[2020-09-09 12:51:29,397] {loghandler.py:20} MainThread INFO in loghandler: filepath /Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input/20.03.2020.pdf, working_dir /Users/kd/Workspace/project/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input/20.03.2020_1

/Users/kd/miniconda3/envs/cv3/lib/python3.5/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


[2020-09-09 12:51:38,412] {loghandler.py:20} MainThread INFO in loghandler: Processing of get_hdfs completed in 1.9235680103302002/13, average per page 0.1479667700254
[2020-09-09 12:51:38,436] {loghandler.py:20} MainThread INFO in loghandler: UnderLineDetection successfully completed


/Users/kd/miniconda3/envs/cv3/lib/python3.5/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


[2020-09-09 12:51:39,841] {loghandler.py:20} MainThread INFO in loghandler: Processing of unify_child_text_blocks completed in 1.4033901691436768/13, average per page 0.10795308993412898


...

[2020-09-09 12:51:42,457] {loghandler.py:20} MainThread INFO in loghandler: Processing of unify_child_text_blocks completed in 2.615550994873047/13, average per page 0.20119623037484977
[2020-09-09 12:51:42,458] {loghandler.py:20} MainThread INFO in loghandler: document structure analysis successfully completed
[2020-09-09 12:51:42,459] {loghandler.py:20} MainThread INFO in loghandler: document structure response started  ===>
[2020-09-09 12:51:42,463] {loghandler.py:20} MainThread INFO in loghandler: Generating response of img_df completed in 0.00048279762268066406
[2020-09-09 12:51:42,464] {loghandler.py:20} MainThread INFO in loghandler: Generating response of table_df completed in 4.0531158447265625e-06
[2020-09-09 12:51:42,465] {loghandler.py:20} MainThread INFO in loghandler: Generating response of line_df completed in 4.0531158447265625e-06
[2020-09-09 12:51:42,500] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.004812955856323242
[2

[2020-09-09 12:51:43,264] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.004554033279418945
[2020-09-09 12:51:43,266] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.019999027252197266
[2020-09-09 12:51:43,294] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.004163980484008789
[2020-09-09 12:51:43,295] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.018624067306518555
[2020-09-09 12:51:43,321] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.0047168731689453125
[2020-09-09 12:51:43,322] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.017956972122192383
[2020-09-09 12:51:43,346] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.0038518905639648438
[2020-09-09 12:51:43,347] {logha

[2020-09-09 12:51:43,949] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.0037779808044433594
[2020-09-09 12:51:43,951] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.01664900779724121
[2020-09-09 12:51:43,974] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.0037050247192382812
[2020-09-09 12:51:43,975] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.015521049499511719
[2020-09-09 12:51:44,000] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.0037140846252441406
[2020-09-09 12:51:44,001] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.01769089698791504
[2020-09-09 12:51:44,002] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.2980010509490967
[2020-09-09 12:51:44,006] {loghandl

[2020-09-09 12:51:44,592] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.03164100646972656
[2020-09-09 12:51:44,615] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.0038750171661376953
[2020-09-09 12:51:44,618] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.0170590877532959
[2020-09-09 12:51:44,641] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.0034699440002441406
[2020-09-09 12:51:44,642] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.016167879104614258
[2020-09-09 12:51:44,667] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.003576993942260742
[2020-09-09 12:51:44,668] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.01645493507385254
[2020-09-09 12:51:44,693] {loghandle

[2020-09-09 12:51:45,297] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.005158185958862305
[2020-09-09 12:51:45,308] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.0035200119018554688
[2020-09-09 12:51:45,309] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.028565168380737305
[2020-09-09 12:51:45,325] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.005923032760620117
[2020-09-09 12:51:45,353] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.0034089088439941406
[2020-09-09 12:51:45,365] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.003932952880859375
[2020-09-09 12:51:45,365] {loghandler.py:20} MainThread INFO in loghandler: Generating response of p_df completed in 0.028359174728393555
[2020-09-09 12:51:45,391] {logha

In [67]:
def draw_bbox_image(draw,page_data):
    
    if page_data['images']:
        try:
            for image_block in page_data['images']:
                    top     = image_block["text_top"];         left   = image_block["text_left"];  
                    bottom  = top+image_block["text_height"];  right  = left+image_block["text_width"]
                    draw.rectangle(((left, top), (right,bottom)), outline='green')
            return draw
        except:
            pass

In [68]:
def draw_bbox_table(draw,page_data):
    
    if page_data['tables']:
        try:
            for table_block in page_data['tables']:
                    top     = table_block["text_top"];         left   = table_block["text_left"];  
                    bottom  = top+table_block["text_height"];  right  = left+table_block["text_width"]
                    draw.rectangle(((left, top), (right,bottom)), outline='blue')
            return draw
        except:
            pass

In [69]:
def draw_bbox_text(draw,page_data):
    
    if page_data['text_blocks']:
        try:
            for text_block in page_data['text_blocks']:
                top     = text_block["text_top"];         left   = text_block["text_left"];  
                bottom  = top+text_block["text_height"];  right  = left+text_block["text_width"]
                draw.rectangle(((left, top), (right,bottom)), outline='red')
            return draw
        except:
            pass

In [70]:
def draw_bbox_pdf(data,image_files,save_path):
    for page_no in range(len(data)):
        image_path = sorted(image_files)[page_no]
        page_data = data[page_no]
        page_width = page_data['page_width']; page_height = page_data['page_height']
        image  = Image.open(image_path)
        image  = image.resize((page_width, page_height))
        draw   = ImageDraw.Draw(image)
        draw  = draw_bbox_text(draw,page_data)
        draw  = draw_bbox_image(draw,page_data)
        draw  = draw_bbox_table(draw,page_data)
        save_filepath = os.path.join(save_path,image_path.split('images/')[1])
        image.save(save_filepath)
            

In [71]:
draw_bbox_pdf(response,pdf_image_paths,save_path)